## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 23rd Mar. 2nd


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-28-2025,Condo/Co-op,231 174th St #1911,Sunny Isles Beach,FL,33160.0,330000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/23...,Beaches MLS,F10485344,N,Y,25.937475,-80.124967


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-28    31
2025-02-27    26
2025-02-24    26
2025-02-26    20
2025-02-25    20
2025-03-02     1
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
65,PAST SALE,2025-02-25,Condo/Co-op,9349 Collins Ave #504,Surfside,FL,33154.0,10600000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11665235,N,Y,25.883000,-80.121840
22,PAST SALE,2025-02-27,Condo/Co-op,4701 N Meridian Ave #503,Miami Beach,FL,33140.0,8075000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11704582,N,Y,25.822813,-80.130733
99,PAST SALE,2025-02-28,Condo/Co-op,900 Brickell Key Blvd Ph 3401,Miami,FL,33131.0,7500000.0,6.0,7.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11662572,N,Y,25.769178,-80.185753
52,PAST SALE,2025-02-27,Condo/Co-op,9701 Collins Ave Unit 1202S,Bal Harbour,FL,33154.0,7150000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11680829,N,Y,25.888491,-80.122881
119,PAST SALE,2025-02-24,Condo/Co-op,16901 Collins Ave #1901,Sunny Isles Beach,FL,33160.0,5685000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11584770,N,Y,25.932764,-80.121036
107,PAST SALE,2025-02-26,Condo/Co-op,330 Sunny Isles Blvd Unit 5-PH04,Sunny Isles Beach,FL,33160.0,3360000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11620390,N,Y,25.929401,-80.126023
13,PAST SALE,2025-02-26,Condo/Co-op,1445 16th St #402,Miami Beach,FL,33139.0,3000000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11720347,N,Y,25.789262,-80.144721
26,PAST SALE,2025-02-24,Condo/Co-op,1415 Sunset Harbour Dr #101,Miami Beach,FL,33139.0,2775000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1415-20t...,MARMLS,A11711696,N,Y,25.795697,-80.143652
56,PAST SALE,2025-02-28,Condo/Co-op,4100 Island Blvd #1004,Aventura,FL,33160.0,2550000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/4100-Island...,MARMLS,A11702941,N,Y,25.941140,-80.135967
17,PAST SALE,2025-02-27,Condo/Co-op,6799 Collins Ave Unit LPH05,Miami Beach,FL,33141.0,2100000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/6799-Collins-A...,MARMLS,A11694464,N,Y,25.852725,-80.120046


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-25,Condo/Co-op,9349 Collins Ave #504,Surfside,FL,33154.0,10600000.0,3.0,3.5,...,-80.121840,https://www.redfin.com/FL/Surfside/9349-Collin...,Ryan Mendell,Maxwell E Realty Inc,None,None,Diane Lieberman,One Sotheby's International Realty,Jill Meyerson-Barzilay,One Sotheby's International Realty
1,PAST SALE,2025-02-27,Condo/Co-op,4701 N Meridian Ave #503,Miami Beach,FL,33140.0,8075000.0,4.0,4.5,...,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Darin Tansey,Douglas Elliman,David Nguah,Douglas Elliman,Darin Tansey,Douglas Elliman,None,None
2,PAST SALE,2025-02-28,Condo/Co-op,900 Brickell Key Blvd Ph 3401,Miami,FL,33131.0,7500000.0,6.0,7.0,...,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Jordan Lederman,Fortune International Realty,Ory Pelyo,Fortune Christie's International Real Estate,Maria Carpeggiani,"MCC Realty, LLC.",None,None
3,PAST SALE,2025-02-27,Condo/Co-op,9701 Collins Ave Unit 1202S,Bal Harbour,FL,33154.0,7150000.0,3.0,4.0,...,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Menachem Fellig,"Compass Florida, LLC.",None,None,Sergio Balinotti,"BRG International , LLC",None,None
4,PAST SALE,2025-02-24,Condo/Co-op,16901 Collins Ave #1901,Sunny Isles Beach,FL,33160.0,5685000.0,4.0,5.5,...,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Jose Augusto Nunes,Algebra Realty.com,Yorghi Abad Mitre,Algebra Realty.com,Solange Santos,"One Way Realty Group, LLC",None,None
5,PAST SALE,2025-02-26,Condo/Co-op,330 Sunny Isles Blvd Unit 5-PH04,Sunny Isles Beach,FL,33160.0,3360000.0,4.0,4.5,...,-80.126023,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,Alexander Goldstein,Miles Goldstein Real Estate,Adi Amuial,Miles Goldstein Real Estate,Mayan Van Der Miller,"Compass Florida, LLC",None,None
6,PAST SALE,2025-02-26,Condo/Co-op,1445 16th St #402,Miami Beach,FL,33139.0,3000000.0,3.0,3.0,...,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Liz Hogan,"Compass Florida, LLC.",None,None,David Blumenfeld,"Method Florida, LLC",None,None
7,PAST SALE,2025-02-24,Condo/Co-op,1415 Sunset Harbour Dr #101,Miami Beach,FL,33139.0,2775000.0,3.0,3.0,...,-80.143652,https://www.redfin.com/FL/Miami-Beach/1415-20t...,Russell Brooke,La Rosa Realty LLC,None,None,Russell Brooke,La Rosa Realty LLC,None,None
8,PAST SALE,2025-02-28,Condo/Co-op,4100 Island Blvd #1004,Aventura,FL,33160.0,2550000.0,3.0,3.5,...,-80.135967,https://www.redfin.com/FL/Aventura/4100-Island...,Nikki Shein,"Williams Island Realty, LLC",None,None,Nikki Shein,"Williams Island Realty, LLC",None,None
9,PAST SALE,2025-02-27,Condo/Co-op,6799 Collins Ave Unit LPH05,Miami Beach,FL,33141.0,2100000.0,2.0,2.0,...,-80.120046,https://www.redfin.com/FL/Miami/6799-Collins-A...,Eileen Nexer,Douglas Elliman,Levente Kovary,Douglas Elliman,Levente Kovary,Douglas Elliman,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

124


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,014,817


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$635


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$125,837,300


In [24]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None

In [31]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PDF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PDF  Last Week Total Dollar Volume
0          124             1014817          635                    125837300.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [34]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
93
Input Previous Week Condo Average Sales Price:
672553
Input Previous Week Condo Average PSF:
528
Input Previous Week Condo Sales total (ex: 198_000_000)
71_200_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [35]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [36]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [37]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [38]:
pd.set_option('display.max_columns',None)

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-25,Condo/Co-op,9349 Collins Ave #504,Surfside,FL,33154.0,10600000.0,3.0,3.5,Fendi Chateau,3378.0,NaN,2016.0,NaN,3138.0,8410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11665235,N,Y,25.883,-80.12184,https://www.redfin.com/FL/Surfside/9349-Collin...,Ryan Mendell,Maxwell E Realty Inc,None,None,Diane Lieberman,One Sotheby's International Realty,Jill Meyerson-Barzilay,One Sotheby's International Realty,1,orange


In [40]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-25,Condo/Co-op,9349 Collins Ave #504,Surfside,FL,33154.0,10600000.0,3.0,3.5,Fendi Chateau,3378.0,NaN,2016.0,NaN,3138.0,8410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11665235,N,Y,25.883000,-80.121840,https://www.redfin.com/FL/Surfside/9349-Collin...,Ryan Mendell,Maxwell E Realty Inc,None,None,Diane Lieberman,One Sotheby's International Realty,Jill Meyerson-Barzilay,One Sotheby's International Realty,1,orange
1,PAST SALE,2025-02-27,Condo/Co-op,4701 N Meridian Ave #503,Miami Beach,FL,33140.0,8075000.0,4.0,4.5,4701 NORTH MERIDIAN CONDO,4226.0,NaN,2019.0,NaN,1911.0,9236.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11704582,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Darin Tansey,Douglas Elliman,David Nguah,Douglas Elliman,Darin Tansey,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-02-28,Condo/Co-op,900 Brickell Key Blvd Ph 3401,Miami,FL,33131.0,7500000.0,6.0,7.0,ASIA CONDOMINIUM,4954.0,NaN,2008.0,NaN,1514.0,7652.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11662572,N,Y,25.769178,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Jordan Lederman,Fortune International Realty,Ory Pelyo,Fortune Christie's International Real Estate,Maria Carpeggiani,"MCC Realty, LLC.",None,None,3,blue
3,PAST SALE,2025-02-27,Condo/Co-op,9701 Collins Ave Unit 1202S,Bal Harbour,FL,33154.0,7150000.0,3.0,4.0,BAL HARBOUR NORTH SOUTH C,3128.0,NaN,2011.0,NaN,2286.0,8074.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11680829,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Menachem Fellig,"Compass Florida, LLC.",None,None,Sergio Balinotti,"BRG International , LLC",None,None,4,blue
4,PAST SALE,2025-02-24,Condo/Co-op,16901 Collins Ave #1901,Sunny Isles Beach,FL,33160.0,5685000.0,4.0,5.5,JADE SIGNATURE CONDO,2980.0,NaN,2018.0,NaN,1908.0,5960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11584770,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Jose Augusto Nunes,Algebra Realty.com,Yorghi Abad Mitre,Algebra Realty.com,Solange Santos,"One Way Realty Group, LLC",None,None,5,blue
5,PAST SALE,2025-02-26,Condo/Co-op,330 Sunny Isles Blvd Unit 5-PH04,Sunny Isles Beach,FL,33160.0,3360000.0,4.0,4.5,PARQUE TOWERS CONDO,3877.0,NaN,2020.0,NaN,867.0,3227.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11620390,N,Y,25.929401,-80.126023,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,Alexander Goldstein,Miles Goldstein Real Estate,Adi Amuial,Miles Goldstein Real Estate,Mayan Van Der Miller,"Compass Florida, LLC",None,None,6,blue
6,PAST SALE,2025-02-26,Condo/Co-op,1445 16th St #402,Miami Beach,FL,33139.0,3000000.0,3.0,3.0,CAPRI SOUTH BEACH CONDO,1842.0,NaN,2008.0,NaN,1629.0,2775.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11720347,N,Y,25.789262,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Liz Hogan,"Compass Florida, LLC.",None,None,David Blumenfeld,"Method Florida, LLC",None,None,7,blue
7,PAST SALE,2025-02-24,Condo/Co-op,1415 Sunset Harbour Dr #101,Miami Beach,FL,33139.0,2775000.0,3.0,3.0,THE THS AT SUNSET HARBOUR,2420.0,NaN,1997.0,NaN,1147.0,2727.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1415-20t...,MARMLS,A11711696,N,Y,25.795697,-80.143652,https://www.redfin.com/FL/Miami-Beach/1415-20t...,Russell Brooke,La Rosa Realty LLC,None,None,Russell Brooke,La Rosa Realty LLC,None,None,8,blue
8,PAST SALE,2025-

In [42]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [43]:
m.save('index.html')

## Data snagger

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [45]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-25,Condo/Co-op,9349 Collins Ave #504,Surfside,FL,33154.0,10600000.0,3.0,3.5,Fendi Chateau,3378.0,NaN,2016.0,NaN,3138.0,8410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11665235,N,Y,25.883,-80.12184,https://www.redfin.com/FL/Surfside/9349-Collin...,Ryan Mendell,Maxwell E Realty Inc,None,None,Diane Lieberman,One Sotheby's International Realty,Jill Meyerson-Barzilay,One Sotheby's International Realty,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [46]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_03032024


# CREATE TEMPLATE 

In [47]:
muni_set = set(df_top_ten['CITY'])

In [48]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [49]:
df_top_ten.reset_index(inplace=True,drop=True)

In [50]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [51]:
top_sale

'Fendi Chateau at 9349 Collins Ave #504 in Surfside'

In [52]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-25,Condo/Co-op,9349 Collins Ave #504,Surfside,FL,33154.0,10600000.0,3.0,3.5,Fendi Chateau,3378.0,NaN,2016.0,NaN,3138.0,8410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11665235,N,Y,25.883000,-80.121840,https://www.redfin.com/FL/Surfside/9349-Collin...,Ryan Mendell,Maxwell E Realty Inc,None,None,Diane Lieberman,One Sotheby's International Realty,Jill Meyerson-Barzilay,One Sotheby's International Realty,1,orange,Fendi Chateau at 9349 Collins Ave #504 in Surf...
1,PAST SALE,2025-02-27,Condo/Co-op,4701 N Meridian Ave #503,Miami Beach,FL,33140.0,8075000.0,4.0,4.5,4701 NORTH MERIDIAN CONDO,4226.0,NaN,2019.0,NaN,1911.0,9236.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11704582,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Darin Tansey,Douglas Elliman,David Nguah,Douglas Elliman,Darin Tansey,Douglas Elliman,None,None,2,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
2,PAST SALE,2025-02-28,Condo/Co-op,900 Brickell Key Blvd Ph 3401,Miami,FL,33131.0,7500000.0,6.0,7.0,ASIA CONDOMINIUM,4954.0,NaN,2008.0,NaN,1514.0,7652.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11662572,N,Y,25.769178,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Jordan Lederman,Fortune International Realty,Ory Pelyo,Fortune Christie's International Real Estate,Maria Carpeggiani,"MCC Realty, LLC.",None,None,3,blue,ASIA CONDOMINIUM at 900 Brickell Key Blvd Ph 3...
3,PAST SALE,2025-02-27,Condo/Co-op,9701 Collins Ave Unit 1202S,Bal Harbour,FL,33154.0,7150000.0,3.0,4.0,BAL HARBOUR NORTH SOUTH C,3128.0,NaN,2011.0,NaN,2286.0,8074.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11680829,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Menachem Fellig,"Compass Florida, LLC.",None,None,Sergio Balinotti,"BRG International , LLC",None,None,4,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
4,PAST SALE,2025-02-24,Condo/Co-op,16901 Collins Ave #1901,Sunny Isles Beach,FL,33160.0,5685000.0,4.0,5.5,JADE SIGNATURE CONDO,2980.0,NaN,2018.0,NaN,1908.0,5960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11584770,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Jose Augusto Nunes,Algebra Realty.com,Yorghi Abad Mitre,Algebra Realty.com,Solange Santos,"One Way Realty Group, LLC",None,None,5,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #190...
5,PAST SALE,2025-02-26,Condo/Co-op,330 Sunny Isles Blvd Unit 5-PH04,Sunny Isles Beach,FL,33160.0,3360000.0,4.0,4.5,PARQUE TOWERS CONDO,3877.0,NaN,2020.0,NaN,867.0,3227.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11620390,N,Y,25.929401,-80.126023,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,Alexander Goldstein,Miles Goldstein Real Estate,Adi Amuial,Miles Goldstein Real Estate,Mayan Van Der Miller,"Compass Florida, LLC",None,None,6,blue,PARQUE TOWERS CONDO at 330 Sunny Isles Blvd Un...
6,PAST SALE,2025-02-26,Condo/Co-op,1445 16th St #402,Miami Beach,FL,33139.0,3000000.0,3.0,3.0,CAPRI SOUTH BEACH CONDO,1842.0,NaN,2008.0,NaN,1629.0,2775.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11720347,N,Y,25.789262,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Liz Hogan,"Compass Florida, LLC.",None,None,David Blumenfeld,"Method Florida, LLC",None,None,7,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St #402 i...
7,PAST SALE,2025-02-24,Condo/Co-op,1415 Sunset Harbour Dr #101,Miami Beac

In [55]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Fendi Chateau closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,100,000 to $10,600,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Sunny Isles Beach, Bal Harbour, Aventura, Miami, Surfside, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 124 condo sales totaling $125,837,300 million from Feb. 23rd to Mar. 2nd. The previous week, brokers closed 93 condo sales totaling $71,200,000.

Last week’s units sold for an average of $1,014,817, higher than the $672,553 average

In [56]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [57]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [58]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-25,Condo/Co-op,9349 Collins Ave #504,Surfside,FL,33154.0,10600000.0,3.0,3.5,Fendi Chateau,3378.0,NaN,2016.0,NaN,3138.0,8410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11665235,N,Y,25.883000,-80.121840,https://www.redfin.com/FL/Surfside/9349-Collin...,Ryan Mendell,Maxwell E Realty Inc,None,None,Diane Lieberman,One Sotheby's International Realty,Jill Meyerson-Barzilay,One Sotheby's International Realty,1,orange,Fendi Chateau at 9349 Collins Ave #504 in Surf...
1,PAST SALE,2025-02-27,Condo/Co-op,4701 N Meridian Ave #503,Miami Beach,FL,33140.0,8075000.0,4.0,4.5,4701 NORTH MERIDIAN CONDO,4226.0,NaN,2019.0,NaN,1911.0,9236.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11704582,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Darin Tansey,Douglas Elliman,David Nguah,Douglas Elliman,Darin Tansey,Douglas Elliman,None,None,2,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
2,PAST SALE,2025-02-28,Condo/Co-op,900 Brickell Key Blvd Ph 3401,Miami,FL,33131.0,7500000.0,6.0,7.0,ASIA CONDOMINIUM,4954.0,NaN,2008.0,NaN,1514.0,7652.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11662572,N,Y,25.769178,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Jordan Lederman,Fortune International Realty,Ory Pelyo,Fortune Christie's International Real Estate,Maria Carpeggiani,"MCC Realty, LLC.",None,None,3,blue,ASIA CONDOMINIUM at 900 Brickell Key Blvd Ph 3...
3,PAST SALE,2025-02-27,Condo/Co-op,9701 Collins Ave Unit 1202S,Bal Harbour,FL,33154.0,7150000.0,3.0,4.0,BAL HARBOUR NORTH SOUTH C,3128.0,NaN,2011.0,NaN,2286.0,8074.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11680829,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Menachem Fellig,"Compass Florida, LLC.",None,None,Sergio Balinotti,"BRG International , LLC",None,None,4,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
4,PAST SALE,2025-02-24,Condo/Co-op,16901 Collins Ave #1901,Sunny Isles Beach,FL,33160.0,5685000.0,4.0,5.5,JADE SIGNATURE CONDO,2980.0,NaN,2018.0,NaN,1908.0,5960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11584770,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Jose Augusto Nunes,Algebra Realty.com,Yorghi Abad Mitre,Algebra Realty.com,Solange Santos,"One Way Realty Group, LLC",None,None,5,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #190...
5,PAST SALE,2025-02-26,Condo/Co-op,330 Sunny Isles Blvd Unit 5-PH04,Sunny Isles Beach,FL,33160.0,3360000.0,4.0,4.5,PARQUE TOWERS CONDO,3877.0,NaN,2020.0,NaN,867.0,3227.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11620390,N,Y,25.929401,-80.126023,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,Alexander Goldstein,Miles Goldstein Real Estate,Adi Amuial,Miles Goldstein Real Estate,Mayan Van Der Miller,"Compass Florida, LLC",None,None,6,blue,PARQUE TOWERS CONDO at 330 Sunny Isles Blvd Un...
6,PAST SALE,2025-02-26,Condo/Co-op,1445 16th St #402,Miami Beach,FL,33139.0,3000000.0,3.0,3.0,CAPRI SOUTH BEACH CONDO,1842.0,NaN,2008.0,NaN,1629.0,2775.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11720347,N,Y,25.789262,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Liz Hogan,"Compass Florida, LLC.",None,None,David Blumenfeld,"Method Florida, LLC",None,None,7,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St #402 i...
7,PAST SALE,2025-02-24,Condo/Co-op,1415 Sunset Harbour Dr #101,Miami Beac

In [60]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Fendi Chateau closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,100,000 to $10,600,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Sunny Isles Beach, Bal Harbour, Aventura, Miami, Surfside, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 124 condo sales totaling $125,837,300 million from Feb. 23rd to Mar. 2nd. The previous week, brokers closed 93 condo sales totaling $71,200,000.

Last week’s units sold for an average of $1,014,817, higher than the $672,553 average

In [68]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Surfside/9349-Collins-Ave-33154/unit-504/home/113411709


In [69]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/4701-N-Meridian-Ave-33140/unit-503/home/79136260


In [63]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/6799-Collins-Ave-33141/unit-LPH05/home/193541016


In [64]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Aventura/4100-Island-Blvd-33160/unit-1004/home/45429226


In [65]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Surfside/9349-Collins-Ave-33154/unit-504/home/113411709


In [66]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/330-Sunny-Isles-Blvd-33160/unit-5-PH04/home/191531924


In [67]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-25,Condo/Co-op,9349 Collins Ave #504,Surfside,FL,33154.0,10600000.0,3.0,3.5,Fendi Chateau,3378.0,NaN,2016.0,NaN,3138.0,8410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11665235,N,Y,25.883000,-80.121840,https://www.redfin.com/FL/Surfside/9349-Collin...,Ryan Mendell,Maxwell E Realty Inc,None,None,Diane Lieberman,One Sotheby's International Realty,Jill Meyerson-Barzilay,One Sotheby's International Realty,1,orange,Fendi Chateau at 9349 Collins Ave #504 in Surf...
1,PAST SALE,2025-02-27,Condo/Co-op,4701 N Meridian Ave #503,Miami Beach,FL,33140.0,8075000.0,4.0,4.5,4701 NORTH MERIDIAN CONDO,4226.0,NaN,2019.0,NaN,1911.0,9236.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11704582,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Darin Tansey,Douglas Elliman,David Nguah,Douglas Elliman,Darin Tansey,Douglas Elliman,None,None,2,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
2,PAST SALE,2025-02-28,Condo/Co-op,900 Brickell Key Blvd Ph 3401,Miami,FL,33131.0,7500000.0,6.0,7.0,ASIA CONDOMINIUM,4954.0,NaN,2008.0,NaN,1514.0,7652.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11662572,N,Y,25.769178,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Jordan Lederman,Fortune International Realty,Ory Pelyo,Fortune Christie's International Real Estate,Maria Carpeggiani,"MCC Realty, LLC.",None,None,3,blue,ASIA CONDOMINIUM at 900 Brickell Key Blvd Ph 3...
3,PAST SALE,2025-02-27,Condo/Co-op,9701 Collins Ave Unit 1202S,Bal Harbour,FL,33154.0,7150000.0,3.0,4.0,BAL HARBOUR NORTH SOUTH C,3128.0,NaN,2011.0,NaN,2286.0,8074.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11680829,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Menachem Fellig,"Compass Florida, LLC.",None,None,Sergio Balinotti,"BRG International , LLC",None,None,4,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
4,PAST SALE,2025-02-24,Condo/Co-op,16901 Collins Ave #1901,Sunny Isles Beach,FL,33160.0,5685000.0,4.0,5.5,JADE SIGNATURE CONDO,2980.0,NaN,2018.0,NaN,1908.0,5960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11584770,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Jose Augusto Nunes,Algebra Realty.com,Yorghi Abad Mitre,Algebra Realty.com,Solange Santos,"One Way Realty Group, LLC",None,None,5,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #190...
5,PAST SALE,2025-02-26,Condo/Co-op,330 Sunny Isles Blvd Unit 5-PH04,Sunny Isles Beach,FL,33160.0,3360000.0,4.0,4.5,PARQUE TOWERS CONDO,3877.0,NaN,2020.0,NaN,867.0,3227.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11620390,N,Y,25.929401,-80.126023,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,Alexander Goldstein,Miles Goldstein Real Estate,Adi Amuial,Miles Goldstein Real Estate,Mayan Van Der Miller,"Compass Florida, LLC",None,None,6,blue,PARQUE TOWERS CONDO at 330 Sunny Isles Blvd Un...
6,PAST SALE,2025-02-26,Condo/Co-op,1445 16th St #402,Miami Beach,FL,33139.0,3000000.0,3.0,3.0,CAPRI SOUTH BEACH CONDO,1842.0,NaN,2008.0,NaN,1629.0,2775.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11720347,N,Y,25.789262,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Liz Hogan,"Compass Florida, LLC.",None,None,David Blumenfeld,"Method Florida, LLC",None,None,7,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St #402 i...
7,PAST SALE,2025-02-24,Condo/Co-op,1415 Sunset Harbour Dr #101,Miami Beac

## Time on Market Calculator

In [71]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 12, 5) ## List (Earlier) date
date2 = datetime(2025, 2, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

84


## Clean CSV for Datawrapper Chart

In [72]:
chart_df = df_top_ten

In [73]:
chart_df = chart_df.fillna(" ")

In [74]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [75]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [76]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [77]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [78]:
chart_df['$/SQUARE FEET'].astype(float)

0    3138.0
1    1911.0
2    1514.0
3    2286.0
4    1908.0
5     867.0
6    1629.0
7    1147.0
8     978.0
9    1400.0
Name: $/SQUARE FEET, dtype: float64

In [79]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [80]:
csv_date_string = today.strftime("%m_%d_%Y")

In [81]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [82]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-25,9349 Collins Ave #504 in Surfside,"$10,600,000",3,3.5,"3,378",2016,"$3,138",MARMLS,A11665235,Ryan Mendell Maxwell E Realty Inc,Diane Lieberman One Sotheby's International Re...
1,2025-02-27,4701 N Meridian Ave #503 in Miami Beach,"$8,075,000",4,4.5,"4,226",2019,"$1,911",MARMLS,A11704582,Darin Tansey Douglas Elliman David Nguah Dougl...,Darin Tansey Douglas Elliman
2,2025-02-28,900 Brickell Key Blvd Ph 3401 in Miami,"$7,500,000",6,7.0,"4,954",2008,"$1,514",MARMLS,A11662572,Jordan Lederman Fortune International Realty O...,"Maria Carpeggiani MCC Realty, LLC."
3,2025-02-27,9701 Collins Ave Unit 1202S in Bal Harbour,"$7,150,000",3,4.0,"3,128",2011,"$2,286",MARMLS,A11680829,"Menachem Fellig Compass Florida, LLC.","Sergio Balinotti BRG International , LLC"
4,2025-02-24,16901 Collins Ave #1901 in Sunny Isles Beach,"$5,685,000",4,5.5,"2,980",2018,"$1,908",MARMLS,A11584770,Jose Augusto Nunes Algebra Realty.com Yorghi A...,"Solange Santos One Way Realty Group, LLC"
5,2025-02-26,330 Sunny Isles Blvd Unit 5-PH04 in Sunny Isle...,"$3,360,000",4,4.5,"3,877",2020,$867,MARMLS,A11620390,Alexander Goldstein Miles Goldstein Real Estat...,"Mayan Van Der Miller Compass Florida, LLC"
6,2025-02-26,1445 16th St #402 in Miami Beach,"$3,000,000",3,3.0,"1,842",2008,"$1,629",MARMLS,A11720347,"Liz Hogan Compass Florida, LLC.","David Blumenfeld Method Florida, LLC"
7,2025-02-24,1415 Sunset Harbour Dr #101 in Miami Beach,"$2,775,000",3,3.0,"2,420",1997,"$1,147",MARMLS,A11711696,Russell Brooke La Rosa Realty LLC,Russell Brooke La Rosa Realty LLC
8,2025-02-28,4100 Island Blvd #1004 in Aventura,"$2,550,000",3,3.5,"2,608",2013,$978,MARMLS,A11702941,"Nikki Shein Williams Island Realty, LLC","Nikki Shein Williams Island Realty, LLC"
9,2025-02-27,6799 Collins Ave Unit LPH05 in Miami Beach,"$2,100,000",2,2.0,"1,500",2008,"$1,400",MARMLS,A11694464,Eileen Nexer Douglas Elliman Levente Kovary Do...,Levente Kovary Douglas Elliman
